In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import re
from requests import get
from urllib import request
from PyPDF2 import PdfReader
import os
import csv
from pykospacing import Spacing
import os.path
import shutil
from tika import parser
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")

target_day = "2014-08-08"

url = f"https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate={target_day}&writeToDate={today}&x=0&y=0&page=1"
temp_url = "https://finance.naver.com/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
    'Referer': 'https://www.naver.com/'
    }

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

last_page = soup.select_one('td.pgRR>a').attrs['href']
temp = "/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2013-05-09&writeToDate=2023-09-01&x=0&y=0&page="
pages = int(last_page.replace(temp,''))

for page in range(1,pages+1):
    page_url = f"https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate={target_day}&writeToDate={today}&x=0&y=0&page={page}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 
        'Referer': 'https://www.naver.com/'
        }

    response = requests.get(page_url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    stock_links = soup.select('table.type_1> tr')

    nomal_page = "https://finance.naver.com/research/"
    print(f"{page} 번 페이지 진행중입니다")
    for link in stock_links:
        taget = link.select("a")
        if taget == []:
            pass
        else:
            url_link = taget[0].attrs['href']
            crawling_link = nomal_page+url_link
            total_url = crawling_link

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 
                'Referer': 'https://www.naver.com/'
                }

            response = requests.get(total_url, headers=headers)
            print(total_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            maket = soup.select_one('p.source').text
            maket = maket.replace("\n",'')
            maket = maket.replace("\t",'')
            maket = maket.split("|")
            maket_name = maket[0]
            date = maket[1]

            stock_title = soup.select_one('th.view_sbj').text

            title = stock_title.replace("\n",'')
            title = title.replace("\t",'')
            title = title.split(maket_name)
            title = title[0]
            title = title.replace('  ','') #리서치 제목
            for idx in range(len(title)): #공백제거
                if title[idx] != ' ':
                    target_idx = idx
            title = title[:target_idx+1]
            new_title = ''
            for i in title:
                if i == "/":
                    new_title += "_"
                else:
                    new_title += i

            title = new_title
            print(f"리서치 제목 : {title}")
            info = soup.select_one("td.view_cnt").text
            info = info.replace('\n','')
            pdf = soup.select("a.con_link")[1].text
            if pdf == '':
                print("PDF 파일이 없습니다 이 경우엔 수집하지 않습니다")
                pass
            else:
                print("PDF 파일이 존재 합니다")
                info = info.split(pdf)
                info = info[0]
                pdf_link = soup.select_one("a.con_link").attrs['href'] #pdf 다운로드 링크
                pdf = soup.select("a.con_link")[1].text #pdf파일 이름
                if "/" in pdf:
                    pdf = pdf.replace('/','_')
                pdf_name = soup.select("a.con_link")[1].text[:-4]
                print(pdf)
                with open(pdf, "wb") as file:   # open in binary mode
                    response = get(pdf_link)               # get request
                    time.sleep(1)
                    file.write(response.content) 

                PDF_FILE_PATH = pdf

                # reader = PdfReader(PDF_FILE_PATH)
                # pages = reader.pages
                # text = ''
                # for page in pages:
                #     sub = page.extract_text()
                #     text += sub
                parsed = parser.from_file(PDF_FILE_PATH)
                text = parsed['content']
                if text is None:
                    print("pdf 파일을 읽을 수 없습니다")
                else:
                    str_text = text
                    new_str = re.sub("\n", "", str_text)
                    new_str = re.sub(" ", "", new_str)

                    spacing = Spacing()
                    kospacing_result = spacing(new_str) 

                    directory = f'./dataset/{date}'

                    if os.path.isdir(directory):
                        print(directory)
                        print("저장경로 있습니다")
                        time.sleep(1)
                    else:
                        print("저장경로가 없습니다")
                        current_path = os.getcwd()
                        os.mkdir(directory)
                        time.sleep(2)
                        if os.path.isdir(directory):
                            print("저장경로 생성하였습니다")
                        else:
                            print("저장경로 생성 실패하였습니다")
                    with open(f'{directory}/{title}_{maket_name}_{date}.text', 'w', encoding='utf-8') as f:
                        f.write('제목\n')
                        f.write(f'{title}\n')
                        f.write('\n')
                        f.write('요약\n')
                        f.write(f'{info}\n')
                        f.write('\n')
                        f.write('내용\n')
                        f.write(f'{kospacing_result}\n')
                    time.sleep(1)
                # 파일 삭제
                remove_file_path = PDF_FILE_PATH # 제거할 파일
                os.remove(remove_file_path)

In [24]:
%pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /private/var/folders/7k/h90v2bm11dn7f71xrf48t7xw0000gn/T/pip-req-build-3ot7pyp5
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /private/var/folders/7k/h90v2bm11dn7f71xrf48t7xw0000gn/T/pip-req-build-3ot7pyp5
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286903 sha256=e29d4d86b0ef103db50cd2af9f33f7c965310301684b6d12bb3ce74797d5b70f
  Stored in directory: /private/var/folders/7k/h90v2bm11dn7f71xrf48t7xw0000gn/T/pip-ephem-wheel-cache-xl7wpvu4/wheels/79/a0/33/16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install numpy

  Using cached numpy-1.24.4-cp38-cp38-macosx_11_0_arm64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-macosx_11_0_arm64.whl (13.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [32]:
%pip install keras

ERROR: Could not find a version that satisfies the requirement keras.layers (from versions: none)
ERROR: No matching distribution found for keras.layers
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tensorflow import T

In [33]:
import tensorflow
tensorflow.__version__

'2.13.0'

In [3]:
from keras.layers import TFSMLayer


ImportError: cannot import name 'TFSMLayer' from 'keras.layers' (/Users/belllaw/Desktop/workspace/BOK_TEAM_1/.conda/lib/python3.8/site-packages/keras/layers/__init__.py)

In [1]:
from pykospacing import Spacing

ImportError: cannot import name 'TFSMLayer' from 'tensorflow.keras.layers' (/Users/belllaw/Desktop/workspace/BOK_TEAM_1/.conda/lib/python3.8/site-packages/keras/api/_v2/keras/layers/__init__.py)

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
from requests import get
from urllib import request
from PyPDF2 import PdfReader
import os
import csv
# from pykospacing import Spacing
import os.path
import shutil
from tika import parser
from datetime import datetime

In [15]:

# 1
# today = datetime.today().strftime("%Y-%m-%d")
today = "2024-08-11"

target_day = "2014-08-11"

url = f"https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate={target_day}&writeToDate={today}&x=0&y=0&page=1"
temp_url = "https://finance.naver.com/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
    'Referer': 'https://www.naver.com/'
    }

In [16]:
# 2
response = requests.get(url, headers=headers)
# res.status_code
soup = BeautifulSoup(response.text, 'html.parser')

# soup.select('td.file')
last_page = soup.select_one('td.pgRR>a').attrs['href']
temp = "/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2014-08-11&writeToDate=2024-08-11&x=0&y=0&page="
pages = int(last_page.replace(temp,''))

In [20]:
#3
for page in range(1, pages+1):
    page_url = f"https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate={target_day}&writeToDate={today}&x=0&y=0&page={page}"

    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
            'Referer': 'https://www.naver.com/'
        }

    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    reports = soup.select('table.type_1 tr')
    normal_page = "https://finance.naver.com/research/"

    print(f"{page} 번 페이지 진행중입니다")

    for report in reports:
        report_link = report.select("a")
        if report_link == []:
            pass
        else:
            report_page = normal_page + report_link[0].attrs['href']  # report_page : 각 레포트별 페이지 url이 반복을 돌면서 하나씩 들어오는 변수
        
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
                'Referer': 'https://www.naver.com/'
                }
            response = requests.get(report_page, headers=headers)
            print(report_page)
            soup = BeautifulSoup(response.text, 'html.parser')

            report_info = soup.select_one('p.source').text
            report_info = report_info.replace("\n",'')
            report_info = report_info.replace("\t",'')
            report_info = report_info.split("|")
            cop_name = report_info[0]
            published_date = report_info[1]
            
            # 리포트 제목 추출
            report_title = soup.select_one('th.view_sbj').text
            title = report_title.replace("\n",'')
            title = title.replace("\t",'')
            title = title.split(cop_name)
            title = title[0]
            title = title.replace('  ','') 
            for idx in range(len(title)): #공백제거
                if title[idx] != ' ':
                    target_idx = idx
            title = title[:target_idx+1]
            new_title = ''
            for i in title:
                if i == "/":
                    new_title += "_"
                else:
                    new_title += i
            title = new_title
            print(f"리서치 제목 : {title}")
            
            # 리포트 전체 내용 추출(PDF 포함)
            content = soup.select_one("td.view_cnt").text
            content = content.replace('\n','')

            # PDF 추출
            pdf_name = soup.select("a.con_link")[1].text
            if pdf_name == '':
                print("PDF 파일이 없습니다 이 경우엔 수집하지 않습니다")
                pass
            else:
                print("PDF 파일이 존재 합니다")
                content = content.split(pdf_name)  # 내용과 PDF 분리
                content = content[0]
                pdf_link = soup.select_one("a.con_link").attrs['href'] # PDF 다운로드 링크
                pdf_name = soup.select("a.con_link")[1].text # PDF 파일 이름  
                if "/" in pdf_name:
                    pdf_name = pdf_name.replace('/','_')
                pdf_name_2 = soup.select("a.con_link")[1].text[:-4]  # .pdf 확장자명 제거
                pdf = pdf_name
                # PDF 저장
                with open(pdf, "wb") as file:   # open in binary mode
                    response = get(pdf_link)  # get request
                    time.sleep(1)
                    file.write(response.content)
                # PDF 텍스트 추출
                PDF_FILE_PATH = pdf
                parsed = parser.from_file(PDF_FILE_PATH)
                text = parsed['content']
                print(text)
                
                if text is None:
                    print("pdf 파일을 읽을 수 없습니다")
                else:
                    str_text = text
                    new_str = re.sub("\n", "", str_text)
                    new_str = re.sub(" ", "", new_str)

                    # spacing = Spacing()
                    # kospacing_result = spacing(new_str)
                    kospacing_result = new_str

                    directory = f'./dataset/{published_date}'

                    if os.path.isdir(directory):
                        print(directory)
                        print("저장경로 있습니다")
                        time.sleep(1)
                    else:
                        print("저장경로가 없습니다")
                        current_path = os.getcwd()
                        os.mkdir(directory)
                        time.sleep(2)
                        if os.path.isdir(directory):
                            print("저장경로 생성하였습니다")
                        else:
                            print("저장경로 생성 실패하였습니다")
                    with open(f'{directory}/{title}_{cop_name}_{published_date}.text', 'w', encoding='utf-8') as f:
                        f.write('제목\n')
                        f.write(f'{title}\n')
                        f.write('\n')
                        f.write('요약\n')
                        f.write(f'{content}\n')
                        f.write('\n')
                        f.write('내용\n')
                        f.write(f'{kospacing_result}\n')
                    time.sleep(1)
                # 파일 삭제
                remove_file_path = PDF_FILE_PATH # 제거할 파일
                os.remove(remove_file_path)

1 번 페이지 진행중입니다
https://finance.naver.com/research/debenture_read.naver?nid=8326&page=1&searchType=writeDate&writeFromDate=2014-08-11&writeToDate=2024-08-11
리서치 제목 : 미국 채권시장, 신규 실업수당 청구 건수가 예상치 하회하며 약세
PDF 파일이 존재 합니다


2024-08-12 15:21:08,226 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /var/folders/7k/h90v2bm11dn7f71xrf48t7xw0000gn/T/tika-server.jar.
2024-08-12 15:21:13,253 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /var/folders/7k/h90v2bm11dn7f71xrf48t7xw0000gn/T/tika-server.jar.md5.
2024-08-12 15:21:14,587 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...







































 

 

 

 

 

 

ResearchCenter  1   

 
 
 

Fixed Income 백윤민 
02_3771_9175 / yoonmin1983@iprovest.com 

 2024.08.09 

Market Comment   Today's Key Chart 
상하  미국 

미국 채권시장, 신규 실업수당 청구 건수가 예상치 하회하며 약세 

■ 전일 동향 

- 전일 미국 채권시장은 약세. 시장금리는 예상치를 하회한 미국 신규 실업수당 

청구건수 등으로 미 경제의 경착륙 우려가 완화되면서 되돌림 지속. 미국 신규 

실업수당 청구건수는 예상치(+24.1만) 를 하회한 23.3만 건을 기록. 여기에 미 국채 

30년물 입찰 응찰률이 2.31배 (12개월 평균 2.40배)를 기록하면서 수요한 약해진 

모습을 이어간 것도 금리 상승 재료로 작용 

 

- 전일 국내 채권시장은 강세. 외국인 3 년 국채선물 매수세가 지속되면서 수요가 

확인되는 가운데, 증시 변동성이 확대되면서 위험자산 회피 심리로 채권 투자심리가 

양호한 모습을 보임. 외국인은 3년물 국채선물을 7,902 계약 순매수 

 

- 전일 크레딧 채권시장은 보합. 국세청이 위메프,티몬 사태 피해 중소기업을 

대상으로 세정 지원을 실시하고 있다는 소식이 전해짐 

■ 금일 전망 

- 금일 채권시장은 약세 예상. 국내 시장금리 레벨 부담이 지속되는 가운데 전일 

미국채 약세 영향을 받으면서 국내 금리도 상승할 것으로 예상 
 

ㅁ 

미 신규 실업수당 청구건수는 23.3만 건을 기록, 예상치 24.1만 건을 하회 

 
   

국내주요금리  (%, bp)  해외주요금리  (%, bp) 
   

 수익률 1D 1M 12M 

국고2YR 3.02 -1.6 -14.0 -69.5 

국고3YR 2.92 -1.2 -20.1 -75.1 

국고5YR 2.94 -1.1 -20.1 -75.1 

국고10YR 3.

KeyboardInterrupt: 

In [36]:
# test
page_url = "https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2014-08-08&writeToDate=2024-08-09&x=0&y=0"

headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
        'Referer': 'https://www.naver.com/'
    }

response = requests.get(page_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

reports = soup.select('table.type_1 tr')
normal_page = "https://finance.naver.com/research/"

print(f"{page} 번 페이지 진행중입니다")

for report in reports:
    report_link = report.select("a")
    if report_link == []:
        pass
    else:
        report_page = normal_page + report_link[0].attrs['href']  # report_page : 각 레포트별 페이지 url이 반복을 돌면서 하나씩 들어오는 변수
        
        headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 
                'Referer': 'https://www.naver.com/'
                }
        response = requests.get(report_page, headers=headers)
        print(report_page)
        soup = BeautifulSoup(response.text, 'html.parser')

        report_info = soup.select_one('p.source').text
        report_info = report_info.replace("\n",'')
        report_info = report_info.replace("\t",'')
        report_info = report_info.split("|")
        cop_name = report_info[0]
        published_date = report_info[1]

        report_title = soup.select_one('th.view_sbj').text
        title = report_title.replace("\n",'')
        title = title.replace("\t",'')
        title = title.split(cop_name)
        title = title[0]
        title = title.replace('  ','') #리서치 제목
        
        for idx in range(len(title)): #공백제거
            if title[idx] != ' ':
                target_idx = idx
        title = title[:target_idx+1]
        new_title = ''
        for i in title:
            if i == "/":
                new_title += "_"
            else:
                new_title += i
                
        title = new_title
        print(f"리서치 제목 : {title}")
        content = soup.select_one("td.view_cnt").text
        content = content.replace('\n','')
        pdf_name = soup.select("a.con_link")[1].text
        if pdf_name == '':
            print("PDF 파일이 없습니다 이 경우엔 수집하지 않습니다")
            pass
        else:
            print("PDF 파일이 존재 합니다")
            content = content.split(pdf_name)  # 내용과 PDF 분리
            content = content[0]
            pdf_link = soup.select_one("a.con_link").attrs['href'] # PDF 다운로드 링크
            pdf_name = soup.select("a.con_link")[1].text # PDF 파일 이름  
            if "/" in pdf_name:
                pdf_name = pdf_name.replace('/','_')
            pdf_name_2 = soup.select("a.con_link")[1].text[:-4]  # .pdf 확장자명 제거
            pdf = pdf_name
            # PDF 저장
            with open(pdf, "wb") as file:   # open in binary mode
                response = get(pdf_link)  # get request
                time.sleep(1)
                file.write(response.content)
            # PDF 텍스트 추출
            PDF_FILE_PATH = pdf
            parsed = parser.from_file(PDF_FILE_PATH)
            text = parsed['content']
            print(text)

            if text is None:
                print("pdf 파일을 읽을 수 없습니다")
            else:
                str_text = text
                new_str = re.sub("\n", "", str_text)
                new_str = re.sub(" ", "", new_str)

                # spacing = Spacing()
                # kospacing_result = spacing(new_str)
                kospacing_result = new_str

                directory = f'./dataset/{published_date}'

                if os.path.isdir(directory):
                    print(directory)
                    print("저장경로 있습니다")
                    time.sleep(1)
                else:
                    print("저장경로가 없습니다")
                    current_path = os.getcwd()
                    os.mkdir(directory)
                    time.sleep(2)
                    if os.path.isdir(directory):
                        print("저장경로 생성하였습니다")
                    else:
                        print("저장경로 생성 실패하였습니다")
                with open(f'{directory}/{title}_{cop_name}_{published_date}.text', 'w', encoding='utf-8') as f:
                    f.write('제목\n')
                    f.write(f'{title}\n')
                    f.write('\n')
                    f.write('요약\n')
                    f.write(f'{content}\n')
                    f.write('\n')
                    f.write('내용\n')
                    f.write(f'{kospacing_result}\n')
                time.sleep(1)
            # 파일 삭제
            remove_file_path = PDF_FILE_PATH # 제거할 파일
            os.remove(remove_file_path)

2 번 페이지 진행중입니다
https://finance.naver.com/research/debenture_read.naver?nid=8326&page=1&searchType=writeDate&writeFromDate=2014-08-08&writeToDate=2024-08-09
리서치 제목 : 미국 채권시장, 신규 실업수당 청구 건수가 예상치 하회하며 약세
PDF 파일이 존재 합니다





































 

 

 

 

 

 

ResearchCenter  1   

 
 
 

Fixed Income 백윤민 
02_3771_9175 / yoonmin1983@iprovest.com 

 2024.08.09 

Market Comment   Today's Key Chart 
상하  미국 

미국 채권시장, 신규 실업수당 청구 건수가 예상치 하회하며 약세 

■ 전일 동향 

- 전일 미국 채권시장은 약세. 시장금리는 예상치를 하회한 미국 신규 실업수당 

청구건수 등으로 미 경제의 경착륙 우려가 완화되면서 되돌림 지속. 미국 신규 

실업수당 청구건수는 예상치(+24.1만) 를 하회한 23.3만 건을 기록. 여기에 미 국채 

30년물 입찰 응찰률이 2.31배 (12개월 평균 2.40배)를 기록하면서 수요한 약해진 

모습을 이어간 것도 금리 상승 재료로 작용 

 

- 전일 국내 채권시장은 강세. 외국인 3 년 국채선물 매수세가 지속되면서 수요가 

확인되는 가운데, 증시 변동성이 확대되면서 위험자산 회피 심리로 채권 투자심리가 

양호한 모습을 보임. 외국인은 3년물 국채선물을 7,902 계약 순매수 

 

- 전일 크레딧 채권시장은 보합. 국세청이 위메프,티몬 사태 피해 중소기업을 

대상으로 세정 지원을 실시하고 있다는 소식이 전해짐 

■ 금일 전망 

- 금일 채권시장은 약세 예상. 국내 시장금리 레벨 부담이 지속되는 가운데 전일 

미국채 약세 영향을 받으면서 국내 금리도 상승할 것으로 예상 
